<a href="https://colab.research.google.com/github/myazann/Lung_Cancer/blob/main/LungCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git init
!git pull https://github.com/myazann/TCIA-API-SDK.git

!cp -r /content/tcia-rest-client-python/src/* /content

!rm -rf tcia-rest-client-java/
!rm -rf tcia-rest-client-python/

!cp /content/drive/MyDrive/Annotation.zip /content
!unzip Annotation.zip
!rm Annotation.zip

!pip install xmltodict

Reinitialized existing Git repository in /content/.git/
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/myazann/TCIA-API-SDK
 * branch            HEAD       -> FETCH_HEAD
Updating 514362c..0ca85ac
Fast-forward
 tcia-rest-client-python/src/tciaclient.py | 24 ++++++++++++++++++++++++
 1 file changed, 24 insertions(+)
Archive:  Annotation.zip
replace Annotation/A0001/1.3.6.1.4.1.14519.5.2.1.6655.2359.102500633407588554681658808214.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [2]:
from tciaclient import TCIAClient
import json
import os
import numpy as np
from xml.etree import cElementTree as ElementTree
import xmltodict
from lxml import etree
import urllib.request, urllib.error, urllib.parse


In [ ]:
tc = TCIAClient(baseUrl="https://services.cancerimagingarchive.net/services/v4", resource = "TCIA")
clctn = "Lung-PET-CT-Dx"
series = json.loads(tc.get_series(collection = clctn, modality = "CT").read())

ptnt_series = {}

for elem in series:
  try:
    if elem["BodyPartExamined"] == "CHEST":
      pt_id = elem["PatientID"].split("-")[1]
      ptnt_series[elem["SeriesInstanceUID"]] = pt_id
  except Exception as e:
    print(e)

In [ ]:
folders = np.array(os.listdir("Annotation"))
folders = [i for i in folders if "A" in i]

bboxs = {}

for folder in folders:
  path = "Annotation/" + folder
  for img in os.listdir(path):
    file_path = path + "/" + img
    try:
      dt = xmltodict.parse(ElementTree.tostring(ElementTree.parse(file_path).getroot()))["annotation"]["object"]["bndbox"]
      img_name = folder + "-" + img.split(".x")[0]
      bboxs[img_name] = dt
    except Exception as e:
      print(e, file_path)

In [4]:
sop = json.loads(tc.get_SOP_instance(list(ptnt_series.keys())[0]).read())

In [13]:
!mkdir deneme

In [15]:
tc.get_single_image(list(ptnt_series.keys())[0], sop[0]["SOPInstanceUID"], downloadPath="./deneme/deneme.dcm")

True

In [28]:
tc.get_image(list(ptnt_series.keys())[0], ".", "deneme.zip")